In [19]:
import os
import random
import seaborn as sns
import csv
import numpy as np
import copy
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pandas as pd
from scipy import stats
%matplotlib inline

file_path="C:/Users/Zeta/Documents/acou_sommeil_HD_ENS/tinnitus-nocturnal-stimulation"

# Importation des données

In [20]:
os.chdir(file_path)
#Recupération des dates des nuits d'intervention pour chaque patient
with open('journees_intervention.csv', 'rt',encoding="utf8") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    dico=[]
    for row in spamreader:
        dico.append(row)

#Suppression de la première ligne de labels
dico=dico[1:]


#Récupération des scores de masquage
with open('MASQUAGE.csv', 'rt',encoding="utf8") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    dico2=[]
    for row in spamreader:
        dico2.append(row)

#Suppression de la première ligne de labels     
dico2=dico2[1:]

#Récupération des données VAS et agenda du sommeil
with open('SUIVI_ACOU_SOMMEIL.csv', 'rt',encoding="ISO-8859-1") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    dico3=[]
    for row in spamreader:
        dico3.append(row)

#Suppression de la première ligne de labels     
dico3=dico3[1:]


# Tris par patient et par instant de mesure, fusion masquage et EVA

In [29]:
#Stockage des jours d'intervention dans le dictionnaire date_passage
date_passage={}

#Creation et initialisation des dataframes de stockage des données triées. 
# Les données seront triées par patient et par instant de mesure (matin ou soir), 
# une dataframe par patient et par instant de mesure
df_matin={}
df_soir={}
df_matin_extended={}
for elm in dico:
    date_passage[elm[0]] = datetime.strptime(elm[1], "%d/%m/%Y")
    df_matin[elm[0]] = pd.DataFrame(columns=['Patient ID', 'relative date', 'masking volume'])
    df_matin_extended[elm[0]] = pd.DataFrame(columns=['Patient ID', 'relative date', 'masking volume',
                                                      "VAS-L","VAS-I","qualite_endormissement","qualite_reveil",
                                                     "forme_veille","heure_mise_au_lit","heure_endormissement",
                                                      "heure_reveil","heure_lever_du_lit"])
    df_soir[elm[0]] = pd.DataFrame(columns=['Patient ID', 'relative date', 'masking volume'])
print(date_passage)

#Tri par patient et par instant de mesure pour le masquage
for elm in dico2:
    if (int(elm[1][:2])>13 or int(elm[1][:2])<4): #mesure du soir, repérée grâce à l'heure de saisie
        df_soir[elm[0]] = df_soir[elm[0]].append(dict(zip(df_soir[elm[0]].columns,[elm[0], (datetime.strptime(elm[-2], "%d/%m/%Y") - date_passage[elm[0]]).days, np.mean([int(elm[2]), int(elm[3]), int(elm[4])])])), ignore_index=True)
    else: #mesures du matin
        df_matin[elm[0]] = df_matin[elm[0]].append(dict(zip(df_matin[elm[0]].columns,[elm[0], (datetime.strptime(elm[-2], "%d/%m/%Y") - date_passage[elm[0]]).days, np.mean([int(elm[2]), int(elm[3]), int(elm[4])])])), ignore_index=True)


    
#Ajout des données de VAS et d'agenda du sommeil
for elm in dico3:
    day = (datetime.strptime(elm[-2], "%d/%m/%Y") - date_passage[elm[0]]).days
    flag = 1
    for index, row in df_matin[elm[0]].iterrows():
        if row['relative date']==day:
            #print(index)
            df_matin_extended[elm[0]] = df_matin_extended[elm[0]].append(dict(zip(df_matin_extended[elm[0]].columns,
            [row['Patient ID'], row['relative date'], row['masking volume'], 
             elm[10][0], elm[11][0], elm[5],elm[6], elm[7], elm[1],elm[2], elm[3],elm[4]]
                                                                                 )), ignore_index=True)
            flag = 0
            break
    if flag == 1 : #le questionnaire suivi a été rempli mais pas le questionnaire de masquage
            
            # On rempli la ligne avec un NaN pour le masquage
            df_matin_extended[elm[0]] = df_matin_extended[elm[0]].append(dict(zip(df_matin_extended[elm[0]].columns,
            [elm[0], day, np.nan, 
             elm[10][0], elm[11][0], elm[5],elm[6], elm[7], elm[1],elm[2], elm[3],elm[4]]
                                                                                 )), ignore_index=True)
            
for elm in df_matin:
    for index, row in df_matin[elm].iterrows():
        flag = 1
        for index2, row2 in df_matin_extended[elm].iterrows():
            if row2["relative date"] == row["relative date"]:
                flag = 0
                break
        if flag == 1: #le questionnaire masquage a été rempli mais pas le questionnaire de suivi
            df_matin_extended[elm] = df_matin_extended[elm].append(dict(zip(df_matin_extended[elm].columns,
            [row['Patient ID'], row['relative date'], row['masking volume'], 
             np.nan, np.nan, np.nan,np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
                                                                         )), ignore_index=True)
    df_matin_extended[elm] = df_matin_extended[elm].sort_values("relative date", axis=0, ascending=True)  

{'1ZN04': datetime.datetime(2019, 10, 15, 0, 0), '1CC05': datetime.datetime(2019, 10, 23, 0, 0), '1PT06': datetime.datetime(2019, 10, 23, 0, 0), '1PI07': datetime.datetime(2019, 10, 26, 0, 0), '1BA07': datetime.datetime(2019, 10, 30, 0, 0), '1MN09': datetime.datetime(2019, 11, 7, 0, 0), '1DL12': datetime.datetime(2019, 11, 15, 0, 0), '1GF14': datetime.datetime(2019, 11, 21, 0, 0), '1SA14': datetime.datetime(2019, 11, 21, 0, 0), '1DA15': datetime.datetime(2019, 11, 29, 0, 0), '1RA17': datetime.datetime(2019, 12, 4, 0, 0), '1MA16': datetime.datetime(2019, 12, 4, 0, 0), '1GB18': datetime.datetime(2019, 12, 11, 0, 0), '1MF19': datetime.datetime(2019, 12, 11, 0, 0)}


# Sauvegarde en csv et affichage contrôle

In [30]:
os.chdir(file_path+"/df_matin")
print("relevés du matin")
for elm in df_matin_extended:
    #Affichage pour une meilleure visualisation de la façon dont les données sont stockées
    print (df_matin_extended[elm])
    #Enregistrement de chaque dataframe dans des fichiers csv comme étape intermédiaire
    df_matin_extended[elm].to_csv(elm+"_matin.csv", sep=";", index=False)

os.chdir(file_path+"/df_soir")
print("")
print("relevés du soir")
for elm in df_soir:
    #Affichage pour une meilleure visualisation de la façon dont les données sont stockées
    print (df_soir[elm])
    #Enregistrement de chaque dataframe dans des fichiers csv comme étape intermédiaire
    df_soir[elm].to_csv(elm+"_soir.csv", sep=";", index=False)

relevés du matin
   Patient ID relative date  masking volume VAS-L VAS-I  \
0       1ZN04           -22        7.333333     2     1   
1       1ZN04           -21        5.000000     3     2   
2       1ZN04           -21        5.000000     3     2   
15      1ZN04           -20        5.666667   NaN   NaN   
3       1ZN04           -19        5.000000     4     3   
4       1ZN04           -18        4.666667     5     3   
5       1ZN04           -17        4.666667     3     2   
6       1ZN04           -16        4.666667     2     2   
7       1ZN04           -15        5.333333     3     3   
8       1ZN04            -3        4.000000     4     2   
9       1ZN04            -1        5.333333     3     2   
10      1ZN04             0        4.666667     3     2   
11      1ZN04             2        4.666667     3     1   
12      1ZN04             3        5.333333     3     2   
13      1ZN04             4        4.333333     3     2   
14      1ZN04             5        4.66

   Patient ID relative date  masking volume
0       1PT06            -9       46.666667
1       1PT06            -8       47.000000
2       1PT06            -7       26.333333
3       1PT06            -6       15.000000
4       1PT06            -5       48.666667
5       1PT06            -4        9.333333
6       1PT06            -3        8.000000
7       1PT06            -2       42.666667
8       1PT06            -1       28.666667
9       1PT06             0       38.000000
10      1PT06             1       43.333333
11      1PT06             2       21.333333
12      1PT06             3        6.666667
13      1PT06             4        5.000000
14      1PT06             5        5.666667
15      1PT06             6       44.333333
   Patient ID relative date  masking volume
0       1PI07            -8       16.666667
1       1PI07            -7       24.333333
2       1PI07            -6       13.000000
3       1PI07            -5       23.333333
4       1PI07            -4     